In [8]:
import ast
import glob

import pandas as pd

pd.set_option("display.max_columns", None)

In [68]:
df = pd.concat([pd.read_csv(i) for i in glob.glob("data/prediction/*.csv")])
df["openai_output"] = df.openai_output.apply(lambda x: ast.literal_eval(x))
print(df.shape)
df.head(2)

(387, 14)


,Job ID,Employer,First Seen Date,Title,Occupation Name,City,State,Job_Type,Education,Description,approx_tokens,openai_output,skills,salary
0,70484,Oracle,2023-01-19,Principal Cloud Software Development Engineer ...,"Software Developers, Applications",Unavailable,California,"Full-Time,Permanent",Bachelor's Degree,About Us Oracle Enterprise Performance Managem...,1406,"([(""Oracle Enterprise Performance Management (...",[('Oracle Enterprise Performance Management (E...,"[('California Pay Range: from $104,100 to $241..."
1,44557,Infosys Technologies,2022-08-28,Java Architect,"Software Developers, Applications",Unavailable,Unavailable,"Full-Time,Permanent",Bachelor's Degree,About Job CTC Undisclosed Job Location United ...,792,"([(""Java Architect"", ""job title""), (""Java"", ""s...","[('Java Architect', 'job title'), ('Java', 'sk...","[('Undisclosed', 'United States of America (US..."


In [69]:
df.to_csv("data/combined_tagged_data.csv", index=False)

In [59]:
df.openai_output[0][0]

0    ([("Oracle Enterprise Performance Management (...
0    ([("Sr Java Developer", "job title"), ("San Jo...
0    ([("C", "knowledge area"), ("XML", "technology...
Name: openai_output, dtype: object

In [60]:
system_prompt = "You are a helpful information extraction system."

skill_template = """Given a job description, your task is to extract all entities and identify their entity types. 
Extract specific entities skills (skills could be knowledge area, tools, technology or personal abilities), job title, education qualification,years of experience,location. 
The output should be in a list of tuples of the following format: [("entity 1", "type of entity 1"), ... ]."""

salary_template = '''Given a job description, extract salary ranges by location, type of compensation it could be base or total or NA. 
In the format [("salary range 1", "location 1", "type of compensation 1"), ...].'''

In [61]:
skill_col = []
for _, row in df.iterrows():
    prompt = system_prompt
    instruction = skill_template
    input_query = str(row["Description"])
    response = str(row["openai_output"][0])

    text = prompt + "### Instruction:\n" + instruction + "\n### Input:\n" + input_query + "\n### Response:\n" + response
    skill_col.append(text)

In [62]:
print(skill_col[0])

You are a helpful information extraction system.### Instruction:
Given a job description, your task is to extract all entities and identify their entity types. 
Extract specific entities skills (skills could be knowledge area, tools, technology or personal abilities), job title, education qualification,years of experience,location. 
The output should be in a list of tuples of the following format: [("entity 1", "type of entity 1"), ... ].
### Input:
About Us Oracle Enterprise Performance Management (EPM) Cloud is an industry-leading SaaS suite of products that helps organizations across the world to model and plan across finance, HR, supply chain, and sales, streamline the financial close process, and drive better decisions. EPM Cloud is deployed in Oracle Cloud Infrastructure (OCI) platform. Our team is responsible for the common software platform on which all EPM Cloud suite of products run. This includes automated provisioning, configuration, deployment, diagnostics, monitoring, cap

In [63]:
salary_col = []
for _, row in df.iterrows():
    prompt = system_prompt
    instruction = salary_template
    input_query = str(row["Description"])
    response = str(row["openai_output"][1])

    text = prompt + "### Instruction:\n" + instruction + "\n### Input:\n" + input_query + "\n### Response:\n" + response
    salary_col.append(text)

In [64]:
print(salary_col[1])

You are a helpful information extraction system.### Instruction:
Given a job description, extract salary ranges by location, type of compensation it could be base or total or NA. 
In the format [("salary range 1", "location 1", "type of compensation 1"), ...].
### Input:
About Job CTC Undisclosed Job Location United States of America (USA) Experience 7 - 10 yrs Description Infosys is seeking a Java Architect. In this position, you will interface with key stakeholders and apply your technical proficiencies across different stages of the Software Development Life CycleSDLC including Requirements Elicitation, Application Architecture definitions & designs through Technologies and Systems' level considerations. You will play an important role in creating the high & low level design artifacts, and also deliver high quality code deliverables. As an Architect, you'll guide team through technical designs' & delivery related challenges, and support activities related to transition, deployments,

In [65]:
df.loc[:, "skill_text"] = skill_col
df.loc[:, "salary_text"] = salary_col
df.head()

,Job ID,Employer,First Seen Date,Title,Occupation Name,City,State,Job_Type,Education,Description,approx_tokens,openai_output,skills,salary,skill_text,salary_text
0,70484,Oracle,2023-01-19,Principal Cloud Software Development Engineer ...,"Software Developers, Applications",Unavailable,California,"Full-Time,Permanent",Bachelor's Degree,About Us Oracle Enterprise Performance Managem...,1406,"([(""Oracle Enterprise Performance Management (...",[('Oracle Enterprise Performance Management (E...,"[('California Pay Range: from $104,100 to $241...",You are a helpful information extraction syste...,You are a helpful information extraction syste...
1,44557,Infosys Technologies,2022-08-28,Java Architect,"Software Developers, Applications",Unavailable,Unavailable,"Full-Time,Permanent",Bachelor's Degree,About Job CTC Undisclosed Job Location United ...,792,"([(""Java Architect"", ""job title""), (""Java"", ""s...","[('Java Architect', 'job title'), ('Java', 'sk...","[('Undisclosed', 'United States of America (US...",You are a helpful information extraction syste...,You are a helpful information extraction syste...
2,36072,IBM,2022-01-20,Sr. Data Architect (Data Lake / Hadoop) – Remo...,"Software Developers, Applications",Unavailable,Unavailable,"Full-Time,Permanent",Bachelor's Degree,"Introduction At IBM, work is more than a job –...",1166,"([(""IBM"", ""company""), (""Data Systems"", ""team"")...","[('IBM', 'company'), ('Data Systems', 'team'),...","[('NA', 'NA', 'NA')]",You are a helpful information extraction syste...,You are a helpful information extraction syste...
3,46296,JPMorgan Chase & Co.,2023-06-27,Software Engineer II-Data Lake,"Software Developers, Applications",Bangalore,Karnātaka,"Full-Time,Permanent",Bachelor's Degree,External Description Str : Join an agile team ...,708,"([(""Software Engineer II"", ""job title""), (""JPM...","[('Software Engineer II', 'job title'), ('JPMo...","[('NA', 'NA', 'NA')]",You are a helpful information extraction syste...,You are a helpful information extraction syste...
4,89749,Unitedhealth Group,2022-11-07,"Senior Software Engineer - Minnetonka, MN","Software Developers, Applications",Hopkins,Minnesota,"Part-Time,Full-Time,Permanent",Bachelor's Degree,Optum is a global organization that delivers c...,988,"([(""Optum"", ""organization""), (""technology"", ""k...","[('Optum', 'organization'), ('technology', 'kn...","[('NA', 'NA', 'NA')]",You are a helpful information extraction syste...,You are a helpful information extraction syste...


In [66]:
train = pd.concat([df[["Job ID", "skill_text"]].rename(columns={"skill_text": "text"}), 
        df[["Job ID", "salary_text"]].rename(columns={"salary_text": "text"})])\
        .reset_index(drop=True)
print(train.shape)
train.head()

(774, 2)


,Job ID,text
0,70484,You are a helpful information extraction syste...
1,44557,You are a helpful information extraction syste...
2,36072,You are a helpful information extraction syste...
3,46296,You are a helpful information extraction syste...
4,89749,You are a helpful information extraction syste...


In [67]:
train.to_csv("data/train.csv", index=False)